#  CIFAR10 photo classification using Keras

The CIFAR-10 small photo classification problem is a standard dataset used in computer vision and deep learning. The problem consists on classifying a dataset containing 60000 images (32x32 pixels) into 10 different categories. We are going to use 50000 images for the training and 10000 images for the test.


<img src="https://miro.medium.com/max/944/1*6XQqOifwnmplS22zCRRVaw.png" width="500">

We start by importing the packages we are going to need

In [5]:
import os
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD, Adam
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import BatchNormalization
from keras.utils import np_utils
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
import tensorflow as tf
K.set_image_dim_ordering('th')

Let's define a list with the classification labels

In [6]:
labels=["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

## Model Training

Let's now load the dataset, define the model and train

In [ ]:
K.clear_session()

#Training parameters
epochs = 1
lrate = 0.01
decay = lrate/epochs
outputpath= "."

# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

# Transform to hot vectors

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

print("We are running on %i classes" % num_classes)

# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(3, 32, 32)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

#Define the optimizer
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

# Compile model
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

# Fit the model
model.fit(X_test, y_test, validation_data=(X_test,y_test), epochs=epochs, batch_size=64)

#Evaluate the model on the test dataset
scores = model.evaluate(X_test, y_test, verbose=10)
print("Accuracy: %.2f%%" % (scores[1]*100))

#Save the model
model.save(os.path.join(outputpath,"model.h5"))

We are running on 10 classes
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 16, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 16, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 16, 16)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 16, 16)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 8, 8)  

## Prediction

Let's now predict on some image using the model that we have just trained

In [ ]:
K.clear_session()

#We load the model we have just trained
model=load_model(os.path.join(outputpath, "model.h5"))

#load local image
img = image.load_img('truck.jpg', target_size=(32,32))
x= image.img_to_array(img)

#Expand the dimensions of the image to match the network architecture
image = numpy.expand_dims(x, axis=0)

#Obtain the index of the highest probability prediction
index_prediction=numpy.argmax(model.predict(image))

#Print the prediction
message= "The image is a " + labels[index_prediction]
if labels[index_prediction]=="airplane" or labels[index_prediction]=="automobile":
    message= "The image is an " + labels[index_prediction]
    
print(message)